<a href="https://colab.research.google.com/github/lavanaythakral/GPT2-answers/blob/master/GPT2%20generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prerequisites

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
import re


TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
!pip install transformers

     |████████████████████████████████| 1.1MB 5.4MB/s 
     |████████████████████████████████| 890kB 29.3MB/s 
     |████████████████████████████████| 3.0MB 33.5MB/s 
     |████████████████████████████████| 1.1MB 46.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f2ebf42d34e7489463c9a315fdd86d51ed6df8ba1a97aa4ec8feeab0c77474c9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
gpt2.mount_gdrive()

Mounted at /content/drive


##GPT2

In [4]:
gpt2.download_gpt2(model_name="355M")
gpt2.copy_checkpoint_from_gdrive(run_name='run1_topical_token')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1_topical_token')

Fetching checkpoint: 1.05Mit [00:00, 270Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 116Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 288Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:08, 160Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 274Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 98.9Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 157Mit/s]                                                       


Loading checkpoint checkpoint/run1_topical_token/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1_topical_token/model-2000


In [6]:
def generate(input):
  print("GPT2 generating for :",input)
  generated_text = gpt2.generate(sess,
              length=100,
              run_name='run1_topical_token',
              return_as_list=True,
              temperature=0.7,
              prefix=input,
              nsamples=15,
              truncate = '.',
              batch_size=5,
              top_k = 5,
              include_prefix = False)
  
  def clean(input_st, sub):
    return input_st.replace(sub, '').lstrip()

  cleaned = []
  for text in generated_text:
    cleaned.append(re.sub(r"^\W+", "",clean(text,'<|endoftext|>')))

  return cleaned


# Data

In [7]:
import json
def load_json(filename):
  with open(filename) as f:
    res = json.load(f)
    return res

In [8]:
pred_file = "/content/drive/My Drive/Thesis project/Thesis project/datasets/post BERT/predictions.json"
ques_file = "/content/drive/My Drive/Thesis project/Thesis project/datasets/post BERT/questions.csv"

In [9]:
df = pd.read_csv(ques_file)
questions = list(df['Questions'])
predictions = load_json(pred_file)

#Generation

## BERT NSP

In [10]:
from torch.nn.functional import softmax
from transformers import BertForNextSentencePrediction, BertTokenizer
model = BertForNextSentencePrediction.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def top_result(seq_A,seq_B):

  response = seq_B[0]
  max_prob = -1
  for seq in seq_B:
    encoded = tokenizer.encode_plus(seq_A, text_pair=seq, return_tensors='pt')
    seq_relationship_logits = model(**encoded)[0]
    probs = softmax(seq_relationship_logits, dim=1)
    if probs[0][0] > max_prob:
      max_prob = probs[0][0]
      response = seq
  return response

In [20]:
def get_response(input):
  temp = generate(input)
  response = top_result(input,temp)
  return response
  
def answers_json(questions,predictions):
  responses = {}
  prefix = "This is beyond my knowledge but here's what I think."
  for idx in range(0,len(questions)):
    print(questions[idx])
    if(str(idx) not in predictions or predictions[str(idx)] == ""):
      res = get_response(questions[idx])
      res = prefix + res
    else:
      res = predictions[str(idx)]
    responses[idx] = res
  return responses

In [14]:
responses = answers_json(questions,predictions)

What is your name?
GPT2 generating for : What is your name?
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
What is SLV?
When is your birthday?
GPT2 generating for : When is your birthday?
Who is Narayan?
GPT2 generating for : Who is Narayan?
Who is your sister?
where are you from?
GPT2 generating for : where are you from?
Where is Pentagon?
where did Trishul take off from?
What is Prithvi?
What is Astra?
Did you have childhood friends?
Which school did you go to as a child?
GPT2 generating for : Which school did you go to as a child?
when did you start liking Science?
GPT2 generating for : when did you start liking Science?
What was your role in Nike-Apache?
What are sounding rockets used for?
Are you religious?
What is SAM?
GPT2 generating for : What is SAM?
Who is Indira Gandhi?
Who conferred the Honorary Doctor of Science degree on to you?
What is TCV?


In [19]:
len(responses)

20

In [18]:
import csv
res_file = "/content/drive/My Drive/Thesis project/Thesis project/datasets/post BERT/responses.csv"
with open(res_file,"w") as f:
  writer = csv.writer(f)
  writer.writerow(["Question","Answer"])
  for idx in range(0,len(questions)):
    writer.writerow([questions[idx], responses[idx]])